### Sample Data Set Generation for 10 Million records

## Please make sure you have a right instance based on your memory needs. For this example ml.c5.4xlarge is used.

In [1]:
import pandas as pd
df = pd.read_csv('../data/seed.csv')

In [2]:
%%time
for i in range(7):
    df = df.append([df]*9, ignore_index=True)

CPU times: user 3.08 s, sys: 2.77 s, total: 5.84 s
Wall time: 5.84 s


In [9]:
df.shape

(10000000, 86)

### Data Transformation

In [6]:
class Counter:
    def __init__(self, seed):
        self.cnt = seed
    def increment(self, x):
        self.cnt += 1
        return str(self.cnt)

In [7]:
%%time
counter = Counter(0)
df["id"] = df["id"].apply(counter.increment)

CPU times: user 3.28 s, sys: 398 ms, total: 3.67 s
Wall time: 3.67 s


In [8]:
%%time
# PLEASE USE YOUR S3 PATH
s3_file_path = 's3://fs-ingest-2/data/10M/seed_10M.csv'
df.to_csv(s3_file_path, index=False)

CPU times: user 3min 40s, sys: 4.31 s, total: 3min 44s
Wall time: 5min 19s


Chunk dataframe and persist in s3

In [10]:
%%time
chunk_rows = 1000000  #num of rows in each chunk, you can change depending on your scenario
s3_bucket = 's3://fs-ingest-2/data/chunks-1M/' # Please use your s3 path
df_list = [df[i:i+chunk_rows] for i in range(0,df.shape[0],chunk_rows)]
for idx, chunk_df in enumerate(df_list):
    s3_file_path = s3_bucket+f'chunk{str(idx)}.csv'
    chunk_df.to_csv(s3_file_path, index=False)

CPU times: user 3min 44s, sys: 4.37 s, total: 3min 48s
Wall time: 5min 29s
